# **Neck Posture with video detection**

In [1]:
!pip install scikit-learn

In [2]:
!pip install keras

  Using cached namex-0.0.8-py3-none-any.whl.metadata (246 bytes)
  Using cached optree-0.12.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (47 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.4 MB/s eta 0:00:00 0:00:01
Using cached namex-0.0.8-py3-none-any.whl (5.8 kB)
Using cached optree-0.12.1-cp311-cp311-macosx_11_0_arm64.whl (283 kB)


In [3]:
!pip install joblib

In [4]:
!pip install seaborn

In [5]:
!pip install opencv-python

In [6]:
import cv2
print(cv2.__version__)

4.10.0


In [7]:
import os
import numpy as np
import mediapipe as mp
import pandas as pd

### Run mediapipe through the pictures to extract the datapoints

In [8]:
# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Paths to dataset folders
good_posture_dir = '/Users/gabrielaclementedeoliveira/Documents/GitHub/final_project_ironhak/pics_ml/good_neck_posture'
bad_posture_dir = '/Users/gabrielaclementedeoliveira/Documents/GitHub/final_project_ironhak/pics_ml/bad_neck_posture'

I0000 00:00:1723719006.061303 1579939 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1723719006.113073 1580393 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1723719006.118495 1580394 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [9]:
# Function to extract key points
def extract_keypoints(landmarks):
    keypoints = []
    for landmark in landmarks.landmark:
        keypoints.extend([landmark.x, landmark.y, landmark.z])
    return keypoints

# Prepare a list to hold the dataset
data = []

# Process images in the "good_neck_posture" folder
for img_name in os.listdir(good_posture_dir):
    img_path = os.path.join(good_posture_dir, img_name)
    image = cv2.imread(img_path)

    # Convert image to RGB for MediaPipe
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the image to extract pose landmarks
    result = pose.process(image_rgb)

    if result.pose_landmarks:
        # Extract key points
        keypoints = extract_keypoints(result.pose_landmarks)
        
        # Append label ("good" = 0)
        keypoints.append(0)
        
        # Add to the dataset list
        data.append(keypoints)

# Process images in the "bad_neck_posture" folder
for img_name in os.listdir(bad_posture_dir):
    img_path = os.path.join(bad_posture_dir, img_name)
    image = cv2.imread(img_path)

    # Convert image to RGB for MediaPipe
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the image to extract pose landmarks
    result = pose.process(image_rgb)

    if result.pose_landmarks:
        # Extract key points
        keypoints = extract_keypoints(result.pose_landmarks)
        
        # Append label ("bad" = 1)
        keypoints.append(1)
        
        # Add to the dataset list
        data.append(keypoints)

# Convert the dataset list to a DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
#df.to_csv('neck_posture_dataset.csv', index=False)

/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


error: OpenCV(4.10.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


### Train the model on the pictures/datapoints

In [ ]:
df.head()

In [ ]:
# Separate features and labels
X = df.iloc[:, :-1].values  # All columns except the last one (key points)
y = df.iloc[:, -1].values   # The last column (labels)

In [ ]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

model = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000))
model.fit(X_train, y_train)

In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'logisticregression__C': [0.01, 0.1, 1, 10, 100],  # Regularization strength
    'logisticregression__solver': ['liblinear', 'saga']  # Solver options
}

# Create a pipeline with scaling and model
pipeline = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000))

# Perform grid search
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)

### Testing another models

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Initialize and train the Gradient Boosting model
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)

# Make predictions
gb_y_pred = gb_model.predict(X_test)

# Evaluate performance
print("Gradient Boosting Accuracy:", accuracy_score(y_test, gb_y_pred))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize and train the Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print("Random Forest Accuracy:", accuracy)

### Ensemble Methods:
Combine multiple models using VotingClassifier to potentially improve performance.

In [ ]:
from sklearn.ensemble import VotingClassifier

# Initialize base models
lr_model = LogisticRegression(max_iter=1000)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)

# Create an ensemble of models
ensemble_model = VotingClassifier(estimators=[
    ('lr', lr_model), 
    ('rf', rf_model), 
    ('gb', gb_model)
], voting='soft')

# Train the ensemble model
ensemble_model.fit(X_train, y_train)

# Make predictions
ensemble_y_pred = ensemble_model.predict(X_test)

# Evaluate performance
print("Ensemble Model Accuracy:", accuracy_score(y_test, ensemble_y_pred))

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
print("Cross-Validation Scores:", scores)
print("Mean Accuracy:", scores.mean())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Assuming you've already trained your ensemble model and made predictions
ensemble_y_pred = ensemble_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, ensemble_y_pred)
print("Ensemble Model Accuracy:", accuracy)

# Compute the confusion matrix
cm = confusion_matrix(y_test, ensemble_y_pred)

# Plot the confusion matrix
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Good', 'Bad'], yticklabels=['Good', 'Bad'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix for Ensemble Model')
plt.show()

# Print classification report
print("Classification Report:\n", classification_report(y_test, ensemble_y_pred, target_names=['Good', 'Bad']))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Make predictions with the best model found by GridSearchCV
grid_y_pred = grid_search.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, grid_y_pred)
print(f"GridSearchCV Model Accuracy: {accuracy * 100:.2f}%")

# Compute the confusion matrix
cm = confusion_matrix(y_test, grid_y_pred)

# Plot the confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Good', 'Bad'], yticklabels=['Good', 'Bad'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix for GridSearchCV Model')
plt.show()

# Print classification report
print("Classification Report:\n", classification_report(y_test, grid_y_pred, target_names=['Good', 'Bad']))

### Saving the model

In [ ]:
import joblib

# Save the trained ensemble model
#joblib.dump(ensemble_model, 'ensemble_model.pkl')

In [ ]:
# Load the trained model
loaded_model = joblib.load('ensemble_model.pkl')

# Make predictions with the loaded model
loaded_y_pred = loaded_model.predict(X_test)

# Evaluate performance
print("Loaded Model Accuracy:", accuracy_score(y_test, loaded_y_pred))

In [ ]:
# Save the best model from GridSearchCV
#joblib.dump(grid_search.best_estimator_, 'best_logistic_model.pkl')

In [ ]:
# Load the saved model
loaded_model = joblib.load('best_logistic_model.pkl')

# Use the model to make predictions
y_pred = loaded_model.predict(X_test)

# Calculate accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

## Video recognition of the neck posture

In [ ]:
# Load the trained ensemble model
model = joblib.load('best_logistic_model.pkl')

# Initialize MediaPipe for pose detection
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Define function to extract key points
def extract_keypoints(landmarks):
    keypoints = []
    for landmark in landmarks.landmark:
        keypoints.extend([landmark.x, landmark.y, landmark.z])
    return keypoints

# Initialize webcam
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    if not ret:
        break
    
    # Convert image to RGB
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Process the image
    result = pose.process(image_rgb)
    
    if result.pose_landmarks:
        # Extract key points
        keypoints = extract_keypoints(result.pose_landmarks)
        
        # Prepare keypoints for prediction
        keypoints_array = np.array(keypoints).reshape(1, -1)
        
        # Predict posture
        prediction = model.predict(keypoints_array)
        
        # Display result and alert
        if prediction[0] == 1:
            cv2.putText(frame, "Bad Posture Alert!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        else:
            cv2.putText(frame, "Good Posture", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    
    # Display the resulting frame
    cv2.imshow('Posture Detection', frame)
    
    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()